In [25]:
from typing import Any
import numpy as np
import matplotlib.pyplot as plt
from numpy import load
import torch

In [26]:
data_path = "../../data/mnist/"
x_test = torch.Tensor(load(f'{data_path}/x_test.npy'))
x_train = torch.Tensor(load(f'{data_path}/x_train.npy'))
y_test = torch.Tensor(load(f'{data_path}/y_test.npy'))
y_train = torch.Tensor(load(f'{data_path}/y_train.npy'))

In [27]:
print(x_train.shape)
print(y_train.shape)

torch.Size([4000, 784])
torch.Size([4000])


In [28]:
def get_random(shape, min_=-0.5, max_=0.5):
    return torch.FloatTensor(*shape).uniform_(min_, max_).requires_grad_()

# Model

```yaml
Generator:
  - Input: 100 (90 noise, 10 one-hot)
  - h1: 150
  - h2: 400
  - h3: 784

FeatureExtractor:
  - Input: 784 (28x28)
  - h1: 400
  - h2: 150
  - h3: 100

Discriminator:
  - Input: 100
  - h1: 50
  - h2: 1

Classifier:
  - Input: 100
  - h1: 50
  - h2: 10
```

In [21]:
class Generator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.w1, self.b1 = get_random((100, 150)), torch.zeros((150, 1))
        self.w2, self.b2 = get_random((150, 400)), torch.zeros((400, 1))
        self.w3, self.gb3 = get_random((400, 784)), torch.zeros((784, 1))
    
    def forward(self, x, **kwargs):
        # TODO: Do the forward.
        return x

In [ ]:
class FeatureExtractor(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.w1, self.b1 = get_random((784, 400)), torch.zeros((400, 1))
        self.w2, self.b2 = get_random((400, 150)), torch.zeros((150, 1))
        self.w3, self.b3 = get_random((150, 100)), torch.zeros((100, 1))
    
    def forward(self, x, **kwargs):
        # TODO: Do the forward.
        return x

In [ ]:
class Discriminator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.w1, self.b1 = get_random((100, 50)), torch.zeros((50, 1))
        self.w2, self.b2 = get_random((50, 1)), torch.zeros((1, 1))
    
    def forward(self, x, **kwargs):
        # TODO: Do the forward.
        return x    

In [ ]:
    
class Classifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.w1, self.b1 = get_random((100, 50)), torch.zeros((50, 1))
        self.w2, self.b2 = get_random((50, 10)), torch.zeros((10, 1))
    
    def forward(self, x, **kwargs):
        # TODO: Do the forward.
        return x


In [22]:
# TODO: Train only classifier to learn how to work with these things.

torch.Size([50, 1])